In [22]:
## Load TEDS-D 2017 Data into a Pandas Data Frame
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import os

## Specify WD
wd = "/home/abf/BINF667013_Final_Project/"
os.chdir(wd)

teds=pd.read_csv("datasets/tedsd_puf_2017.csv")
teds

## Convert '-9' to NA
teds=teds.replace({-9:np.nan})

### Count the number of complete cases in this dataset

In [ ]:
teds.dropna().shape

### Count the number of cases with valid observations for at least 50% of available measures

In [ ]:
mask=teds.apply(lambda x: np.isnan(x)).apply(lambda x: sum(x) < 39, axis=1)
teds[mask].shape

### Count the number of cases with valid observations for at least 75% of available measures

In [ ]:
mask=teds.apply(lambda x: np.isnan(x)).apply(lambda x: sum(x) < 19, axis=1)
teds[mask].shape

### Count the number of cases with valid observations for at least 90% of available measures

In [ ]:
mask=teds.apply(lambda x: np.isnan(x)).apply(lambda x: sum(x) < 8, axis=1)
teds[mask].shape

### Explore Correlation between missing values

In [ ]:
import missingno as msno
msno.heatmap(teds, fontsize=12)

In [ ]:
msno.dendrogram(teds)

### Restrict data to records that are at least 75% complete

In [23]:
mask=teds.apply(lambda x: np.isnan(x)).apply(lambda x: sum(x) < 19, axis=1)
print(teds[mask].shape)
teds = teds[mask]

## make a subset for development
#teds=teds.iloc[0:16300, ]

(1308896, 76)


### Convert Observations to Categorical Variables

In [3]:
for col in teds.columns:
    if not (col =="CASEID" or col == "DISYR"):
        teds[col] = teds[col].astype('category')

### Use MissForest implemented in missingpy to impute missing values

In [26]:
## Stupid hack due to broken module name #WastingMyTime #SloppyCode
import sys
import sklearn.neighbors
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
from os.path import exists

def data_frame_chunks(df, n):
    for i in range(0, df.shape[0], n):
        
        # Get range of records in df
        low=i
        high=min(i + n, df.shape[0])
        
        # Return Iterator for data frame chunks
        yield df.iloc[low:high,]

def impute_missforest(df, seed=123):
    imputer = MissForest(
        random_state=seed,
        criterion=('squared_error', 'gini'), # Package mispecifies criterion
        n_jobs=-1
    )
    X=imputer.fit_transform(
        df.drop(columns=['CASEID', 'DISYR']),
        cat_vars=np.array([x for x in range(0,74)])
    )

    dg=pd.DataFrame(X)
    dg.columns=df.columns[2:76]
    return(dg)

dflist = [impute_missforest(c) for c in chunks]
teds_imputed = pd.concat(dflist)
teds_imputed.to_csv(
    path_or_buf="analysis_objects/teds_imputed.csv"
)




In [27]:
for i in range(0, len(dflist)):
    print(dflist[i].shape)
    print(dflist[i].index)

(100000, 76)
Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            221619, 221624, 221625, 221626, 221627, 221628, 221629, 221630,
            221632, 221633],
           dtype='int64', length=100000)
(100000, 76)
Int64Index([221634, 221636, 221640, 221641, 221643, 221644, 221645, 221646,
            221648, 221651,
            ...
            364742, 364743, 364744, 364745, 364746, 364747, 364748, 364749,
            364750, 364751],
           dtype='int64', length=100000)
(100000, 76)
Int64Index([364754, 364756, 364757, 364758, 364759, 364760, 364761, 364762,
            364763, 364764,
            ...
            471820, 471821, 471822, 471823, 471824, 471825, 471826, 471827,
            471828, 471829],
           dtype='int64', length=100000)
(100000, 76)
Int64Index([471830, 471831, 471832, 471833, 471834, 471835, 471836, 471837,
            471838, 471839,
            ...
            572529, 572530, 57

In [15]:
df=pd.DataFrame()
df.index=pd.RangeIndex(start=0, stop=100, step=1)

print(df)

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]

[100 rows x 0 columns]
